In [39]:
#****************
#change all these except for BUCKET_LOC
#****************
PROJECT_ID = 'ml-demo-384110'
DATASET_ID = 'bqml_tutorial_us'
OJBECT_TABLE_NAME= 'obj_cloud_ai_images'
CONN_NAME= 'conn_cloud_ai'
CONN_SERVICE_ACCOUNT = ""
REGION = "US"
VISION_MODEL_NAME = 'bqml-cloud-ai-vision'
TRANSLATE_MODEL_NAME = 'bqml-cloud-ai-translate'
TEXT_MODEL_NAME = 'bqml-cloud-ai-text'
LLM_TEXT_MODEL_NAME = 'bqml-llm_model'



Demo of BQML cloud-ai API functionality.

Objective of Demo:  Using unstructured data in BQ Data Analytics workflows

Main steps:
1.   create object table
2.   Call vision ai (ML.ANNOTATE_IMAGE) to do OCR
3.   Call translate ai (ML.TRANSLATE) to detect language of text
4.   Call translate ai (ML.TRANSLATE) to translate non-english records to english
5.   Call NLP ai (ML.UNDERSTAND_TEXT) to classify text
6.   Call text-bison LLM model (ML.GENERATE_TEXT) more detail here: https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-text



In [12]:
!pip install google-cloud-bigquery-connection
from google.cloud import bigquery, bigquery_connection_v1

In [3]:
!gcloud config set project {PROJECT_ID}
shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID = shell_output[0]
print("Project ID now set to:", PROJECT_ID)

Updated property [core/project].
Project ID now set to: ml-demo-384110


In [4]:
#you may need to enable apis
!gcloud services enable vision.googleapis.com language.googleapis.com

Operation "operations/acat.p2-1008225662928-764e721b-65bb-43ff-9a67-7d0ba5c21622" finished successfully.


In [26]:
#Create BQ Connection
from google.cloud import bigquery as bq
from google.cloud import bigquery_connection_v1 as bq_connection
import googleapiclient.discovery

client = bq_connection.ConnectionServiceClient()
new_conn_parent="projects/{}/locations/{}".format(PROJECT_ID, "US")
exists_conn_parent="projects/{}/locations/{}/connections/{}".format(PROJECT_ID, "US", CONN_NAME)
cloud_resource_properties = bq_connection.CloudResourceProperties({})

try:
  request = client.get_connection(request=bq_connection.GetConnectionRequest(name=exists_conn_parent))
  CONN_SERVICE_ACCOUNT = "serviceAccount:{}".format(request.cloud_resource.service_account_id)
except Exception as e:
  connection = bq_connection.types.Connection(
          {
              "friendly_name": CONN_NAME,
              "cloud_resource": cloud_resource_properties
          }
      )
  request = bq_connection.CreateConnectionRequest(
          {
              "parent": new_conn_parent,
              "connection_id": CONN_NAME,
              "connection": connection
          }
      )
  response = client.create_connection(request)
  CONN_SERVICE_ACCOUNT = "serviceAccount:{}".format(response.cloud_resource.service_account_id)
print(CONN_SERVICE_ACCOUNT)

serviceAccount:bqcx-1008225662928-osik@gcp-sa-bigquery-condel.iam.gserviceaccount.com


In [27]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT} --role='roles/serviceusage.serviceUsageConsumer'

In [28]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT} --role='roles/bigquery.connectionUser'

In [29]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT}  --role='roles/cloudtranslate.user'

In [31]:
#create object table
try:
    client = bigquery.Client(project=PROJECT_ID)
    sql = f"""
          CREATE OR REPLACE EXTERNAL TABLE
          `{PROJECT_ID}.{DATASET_ID}.{OJBECT_TABLE_NAME}`
          WITH CONNECTION `{REGION}.{CONN_NAME}` OPTIONS( object_metadata = 'SIMPLE',
          uris = ['gs://data-analytics-demos/TextOCR_images/*']);
          """ 
        
    query_job = client.query(sql)
    results = query_job.result()

except Exception as e:
    raise Exception(str(e))



In [33]:
#query object table
try:
    client = bigquery.Client(project=PROJECT_ID)
    sql = f"""
      SELECT * FROM
      `{PROJECT_ID}.{DATASET_ID}.{OJBECT_TABLE_NAME}` limit 50 ;
      """
  
    df = client.query(sql).to_dataframe()

except Exception as e:
    raise Exception(str(e))

df.head(6)

,uri,generation,content_type,size,md5_hash,updated,metadata
0,gs://data-analytics-demos/TextOCR_images/00875...,1680039280565689,image/jpeg,766494,b20ee2f0609a75e4ba95666893a1ba8c,2023-03-28 21:34:40.607000+00:00,[]
1,gs://data-analytics-demos/TextOCR_images/01023...,1680039280553219,image/jpeg,824430,51911a441d0213f3f644b4c092ceb137,2023-03-28 21:34:40.605000+00:00,[]
2,gs://data-analytics-demos/TextOCR_images/0328f...,1680039280563977,image/jpeg,747167,dec21a8a7cb426272eb7cf2739cfde06,2023-03-28 21:34:40.605000+00:00,[]
3,gs://data-analytics-demos/TextOCR_images/04b03...,1680039280568388,image/jpeg,702377,87f32af89141a6269501dd285ac6699c,2023-03-28 21:34:40.608000+00:00,[]
4,gs://data-analytics-demos/TextOCR_images/07901...,1680039280574171,image/jpeg,851695,16886737338012a0e556e8331eba7f8a,2023-03-28 21:34:40.609000+00:00,[]
5,gs://data-analytics-demos/TextOCR_images/08d54...,1680039280676556,image/jpeg,815716,427c6695e732898c8606c1ec2f21cf8f,2023-03-28 21:34:40.708000+00:00,[]


In [41]:
#create vision api model
try:
    client = bigquery.Client(project=PROJECT_ID)
    sql = """
      CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET_ID}.{VISION_MODEL_NAME}`
      REMOTE WITH CONNECTION `{PROJECT_ID}.{REGION}.{CONN_NAME}`
      OPTIONS (
        remote_service_type = 'cloud_ai_vision_v1');
      """.format(PROJECT_ID = PROJECT_ID,DATASET_ID = DATASET_ID, OJBECT_TABLE_NAME=OJBECT_TABLE_NAME, VISION_MODEL_NAME=VISION_MODEL_NAME, REGION=REGION, CONN_NAME=CONN_NAME)
    query_job = client.query(sql)
    results = query_job.result()

    model_id = '{}.{}.{}'.format(PROJECT_ID, DATASET_ID, VISION_MODEL_NAME)

    model = client.get_model(model_id)  # Make an API request.

    full_model_id = "{}.{}.{}".format(model.project, model.dataset_id, model.model_id)
    friendly_name = model.friendly_name
    print(
      "Vision Model now available '{}'".format(full_model_id, friendly_name)
  )
except Exception as e:
    raise Exception(str(e))


Vision Model now available 'ml-demo-384110.bqml_tutorial_us.bqml-cloud-ai-vision'


In [35]:
#create translate api model
try:
    client = bigquery.Client(project=PROJECT_ID)
    sql = """
      CREATE OR REPLACE MODEL
      `{}.{}.{}` REMOTE
      WITH CONNECTION `{}.{}.{}`
      OPTIONS ( remote_service_type = 'CLOUD_AI_TRANSLATE_V3');
      """.format(PROJECT_ID,DATASET_ID, TRANSLATE_MODEL_NAME,PROJECT_ID, REGION, CONN_NAME)
    query_job = client.query(sql)
    results = query_job.result()

    model_id = '{}.{}.{}'.format(PROJECT_ID, DATASET_ID, TRANSLATE_MODEL_NAME)

    model = client.get_model(model_id)  # Make an API request.

    full_model_id = "{}.{}.{}".format(model.project, model.dataset_id, model.model_id)
    friendly_name = model.friendly_name
    print(
    "Translate Model now available '{}'".format(full_model_id, friendly_name)
  )

except Exception as e:
    raise Exception(str(e))

Translate Model now available 'ml-demo-384110.bqml_tutorial_us.bqml-cloud-ai-translate'


In [36]:
#create NLP model
try:
    client = bigquery.Client(project=PROJECT_ID)
    sql = """
      CREATE OR REPLACE MODEL
      `{}.{}.{}` REMOTE
      WITH CONNECTION `{}.{}.{}`
      OPTIONS ( remote_service_type = 'cloud_ai_natural_language_v1');
      """.format(PROJECT_ID,DATASET_ID, TEXT_MODEL_NAME,PROJECT_ID, REGION, CONN_NAME)
    query_job = client.query(sql)
    results = query_job.result()

    model_id = '{}.{}.{}'.format(PROJECT_ID, DATASET_ID, TEXT_MODEL_NAME)

    model = client.get_model(model_id)  # Make an API request.

    full_model_id = "{}.{}.{}".format(model.project, model.dataset_id, model.model_id)
    friendly_name = model.friendly_name
    print(
    "NLP Model now available '{}'".format(full_model_id, friendly_name)
  )

except Exception as e:
    raise Exception(str(e))

NLP Model now available 'ml-demo-384110.bqml_tutorial_us.bqml-cloud-ai-text'


In [37]:
#Use vision ai to get text from images
#then use translate ai to check language of text
try:
    client = bigquery.Client(project=PROJECT_ID)
    sql = """
        CREATE OR REPLACE TABLE
          `{}.{}.temp_image_results` AS
        SELECT
          --this column must be aliased/named 'text_content'
          STRING(ml_annotate_image_result.full_text_annotation.text) AS text_content
        FROM
          ML.ANNOTATE_IMAGE( MODEL `{}.{}.{}`,
            TABLE `{}.{}.{}`,
            STRUCT(['TEXT_DETECTION'] AS vision_features));

        CREATE OR REPLACE TABLE {}.{}.temp_detect_language_results as
        SELECT
          ml_translate_result.languages[0].language_code,
          ml_translate_result.languages[0].confidence, *
        FROM
          ML.TRANSLATE( MODEL `{}.{}.{}`, TABLE `{}.{}.temp_image_results`, STRUCT("detect_language" AS translate_mode));

        SELECT * FROM {}.{}.temp_detect_language_results;
        """.format(PROJECT_ID, DATASET_ID, PROJECT_ID, DATASET_ID, VISION_MODEL_NAME, PROJECT_ID, DATASET_ID, OJBECT_TABLE_NAME, PROJECT_ID, DATASET_ID, PROJECT_ID, DATASET_ID, TRANSLATE_MODEL_NAME, PROJECT_ID,DATASET_ID, PROJECT_ID, DATASET_ID )

    df = client.query(sql).to_dataframe()

except Exception as e:
    raise Exception(str(e))



df.head(6)

,language_code,confidence,ml_translate_result,ml_translate_status,text_content
0,"""ar""",1,"{""languages"":[{""confidence"":1,""language_code"":...",,فة، لم يكن حولها جدار\nبا دور للناس، وإنما كان...
1,"""de""",0.88780957,"{""languages"":[{""confidence"":0.88780957,""langua...",,TI\nVifebook!\n9\noh\nhein!\nDIR\nfreice Tag: ...
2,"""de""",1,"{""languages"":[{""confidence"":1,""language_code"":...",,Alleinige Inseraten-Annahme\nbei Rudolf Mosse\...
3,"""de""",0.39134026,"{""languages"":[{""confidence"":0.39134026,""langua...",,"Rainhard Fendrich Rainhard Fendrich, Katie Mel..."
4,"""de""",0.96768254,"{""languages"":[{""confidence"":0.96768254,""langua...",,PRICKELMA\nCLUB-MATE\nSCHE\nMATE\nDe in des\nw...
5,"""de""",0.29555309,"{""languages"":[{""confidence"":0.29555309,""langua...",,O\n12\nΩ\nOMEGA\nSeamaster\nPROFESSIONAL\nAXIA...


In [ ]:
#use translate ai to translate text
try:
    client = bigquery.Client(project=PROJECT_ID)
    sql = """
        SELECT text_content, language_code,
          ml_translate_result.translations[0].translated_text
        FROM
          ML.TRANSLATE( MODEL `{}.{}.{}`, TABLE `{}.{}.temp_detect_language_results`, STRUCT("translate_text" as translate_mode, "en" as target_language_code));
        """.format( PROJECT_ID,DATASET_ID, TRANSLATE_MODEL_NAME, PROJECT_ID,DATASET_ID)

    df = client.query(sql).to_dataframe()

except Exception as e:
    raise Exception(str(e))

df.head(6)

,text_content,language_code,translated_text
0,فة، لم يكن حولها جدار\nبا دور للناس، وإنما كان...,"""ar""","""There was no wall around it with no role for ..."
1,PRICKELMA\nCLUB-MATE\nSCHE\nMATE\nDe in des\nw...,"""de""","""PRICKELMA CLUB-MATE SCHE MATE De in des wwwww..."
2,F FÜR TRÄUME\nFortsetzung von Seite 304\nDie K...,"""de""","""F FOR DREAMS Continued from page 304 The cham..."
3,O\n12\nΩ\nOMEGA\nSeamaster\nPROFESSIONAL\nAXIA...,"""de""","""O 12Ω OMEGA Seamaster PROFESSIONAL AXIAL HOME..."
4,Alleinige Inseraten-Annahme\nbei Rudolf Mosse\...,"""de""","""Sole acceptance of advertisements at Rudolf M..."
5,TI\nVifebook!\n9\noh\nhein!\nDIR\nfreice Tag: ...,"""de""","""TI Vibook! 9 oh hey! DIR freice day: Think ™R..."
6,"Rainhard Fendrich Rainhard Fendrich, Katie Mel...","""de""","""Rainhard Fendrich Rainhard Fendrich, Katie Me..."
7,Reaktionen im Netz\nLifestream Termine Die Ini...,"""de""","""Reactions on the web Lifestream Dates The ini..."
8,WÜNSCHEN SIE ERSTKLASSIGE\nTAPETEN\nDER NEUEN ...,"""de""","""WANT NEW SEASON PREMIUM WALLPAPERS. HOW TO AS..."
9,THE CLOISTERS WETLAND\nThe wetland you see her...,"""en""","""THE CLOISTERS WETLAND\nThe wetland you see he..."


In [38]:
#create object table
try:
    client = bigquery.Client(project=PROJECT_ID)
    sql = """
      SELECT
      text_content AS `Original Input`,
      replace(string(ml_understand_text_result.categories[0].name), '/','') AS `Classified Name`,
      ml_understand_text_result.categories[0].confidence AS `Confidence`
      FROM
      ML.UNDERSTAND_TEXT( MODEL `{}.{}.{}`,
        TABLE `{}.{}.temp_detect_language_results`,
        STRUCT('classify_text' AS nlu_option))
        where  ml_understand_text_result.categories[0].name is not null ;
      """.format(PROJECT_ID,DATASET_ID, TEXT_MODEL_NAME, PROJECT_ID,DATASET_ID )
    df = client.query(sql).to_dataframe()

except Exception as e:
    raise Exception(str(e))

df.head()


,Original Input,Classified Name,Confidence
0,try factory where UN peacekeepers were sta-\nt...,Sensitive Subjects,0.80000001
1,THE CORSICAN CAPER\nlove at first bite. Elena ...,Food & DrinkCooking & Recipes,0.56
2,GIANTS\n軍\nShawn Estes\nSAN FRANCISCO GIANTS: ...,SportsTeam SportsBaseball,0.98000002
3,Bryce Canyon National Park\nLodge\n♦\n0.5 mi\n...,TravelTourist DestinationsRegional Parks & Gar...,0.95999998
4,For ticket information FM\nAnnual\nAf\nommage ...,Arts & EntertainmentMusic & AudioClassical Music,0.91000003


In [50]:
#Use vision ai to get text from images
#then use translate ai to check language of text
try:
    client = bigquery.Client(project=PROJECT_ID)
    sql = f"""

CREATE OR REPLACE TABLE
 `{DATASET_ID}.temp_summarize` 
 AS
SELECT
  ml_generate_text_result['predictions'][0]['content'] AS generated_text,
  ml_generate_text_result['predictions'][0]['safetyAttributes'] AS safety_attributes,
  ml_generate_text_result['predictions'][0]['safetyAttributes']['blocked'] AS blocked,
  
  * EXCEPT (ml_generate_text_result)
FROM
  ML.GENERATE_TEXT( MODEL `{DATASET_ID}.llm_model`,
    (
    SELECT
      CONCAT('summarize: ', text_content) AS prompt,
      *
    FROM
      {DATASET_ID}.temp_image_results
    LIMIT
      5 ),
    STRUCT( 0.2 AS temperature,
      100 AS max_output_tokens));




SELECT  *
 FROM `{DATASET_ID}.temp_summarize` 
 
 """

    df = client.query(sql).to_dataframe()

except Exception as e:
    raise Exception(str(e))



df.head(6)

,generated_text,safety_attributes,blocked,ml_generate_text_status,prompt,text_content
0,"""The product is Coca-Cola Zero. It is a zero c...","{""blocked"":false,""categories"":[""Health"",""Illic...",false,,summarize: Ica-Colica Collca-Colca Col\nzero z...,Ica-Colica Collca-Colca Col\nzero zero zero ze...
1,"""sway The data is about a company's business. ...","{""blocked"":false,""categories"":[""Finance"",""Lega...",false,,summarize: Airport\nBusiness\n기\nunagroup\nCen...,Airport\nBusiness\n기\nunagroup\nCentre\nPlymou...
2,"""This is an article about Playmate of the Year...","{""blocked"":false,""categories"":[""Death, Harm & ...",false,,summarize: COMPANY B\n126M AGM 505 IME VE\nAbo...,COMPANY B\n126M AGM 505 IME VE\nAbove: Aboord ...
3,"""This is a book cover for a mystery novel call...","{""blocked"":false,""categories"":[""Finance"",""Illi...",false,,summarize: DELL\nD292\nTWO MIKE SHAYNE\nMYSTER...,DELL\nD292\nTWO MIKE SHAYNE\nMYSTERIES\nDEAD\n...
4,"""""","{""blocked"":true}",true,,summarize: F FÜR TRÄUME\nFortsetzung von Seite...,F FÜR TRÄUME\nFortsetzung von Seite 304\nDie K...
